In [158]:
import pandas  as pd
import numpy as np

In [159]:
relevant_genes = pd.read_csv("DaniDatasets/relevant_genes.csv")

In [160]:
microbiome = pd.read_csv("DaniDatasets/BLCA_csv/microbiome_log_rna.csv",index_col=0)
mirna = pd.read_csv("DaniDatasets/BLCA_csv/miRNA.csv",index_col=0)
svs = pd.read_csv("DaniDatasets/BLCA_csv/svs.csv", index_col=0)
mutations = pd.read_csv("DaniDatasets/BLCA_csv/mutations.csv",index_col=0)
rna_median = pd.read_csv("DaniDatasets/BLCA_csv/RNA_expression_median.csv", index_col=0)
rppa_levels = pd.read_csv("DaniDatasets/BLCA_csv/rppa_levels.csv", index_col=0)
metilation = pd.read_csv("DaniDatasets/BLCA_csv/methylations.csv", index_col=0)
cna_log2 = pd.read_csv("DaniDatasets/BLCA_csv/cna_log2.csv", index_col=0)

In [161]:
for omic in [mutations, metilation, rna_median, cna_log2]:
    columns_to_drop = []
    for column_name in mutations.columns:
        if not (relevant_genes['Gene'] == column_name).any():
            columns_to_drop.append(column_name)
    print("dropped {} columns".format(len(columns_to_drop)))
    omic.drop(columns=columns_to_drop, inplace=True)
    

dropped 0 columns
dropped 0 columns
dropped 0 columns
dropped 0 columns


In [162]:
## svs has a particular configuration
columns_to_drop = []
for column_name in svs.columns:
    gene1, gene2 = column_name.split('_')
    # Check if either gene is in genes_df
    if not ((relevant_genes['Gene'] == gene1).any() 
            or (relevant_genes['Gene'] == gene2).any()):
        columns_to_drop.append(column_name)

# Dropping
svs.drop(columns=columns_to_drop, inplace=True)
svs.shape

(332, 377)

In [163]:
# check for index naming
for i in range(0,len(svs.index)):
    if svs.index[i] != mutations.index[i]:
        print("Concatenation not Correct")
alterations = pd.concat([svs, mutations], axis=1)

In [164]:
# set unique column name
for omic, name in zip([rna_median, metilation, mirna, cna_log2, rppa_levels, alterations, microbiome],
                      ['rna_median', 'metilation', 'mirna', 'cna_log2', 'rppa_levels', 'alterations', 'microbiome']):
    omic.columns = [f"{name}_{c}" for c in omic.columns]

In [165]:
clinical_total = pd.read_csv("DaniDatasets/BLCA_csv/clinical_data.csv", index_col=1)

In [166]:
clinical = clinical_total[['AGE', 'GENDER', 'OS_STATUS', 'OS_MONTHS']]

In [167]:
def convert_status(status):
    if status == '1:DECEASED':
        return 1
    elif status == '0:LIVING':
        return 0
    else:
        print("ERROR: y cannot be empty")
        return np.NaN

def convert_gender(gender):
    if gender == 'Female':
        return 1
    elif gender == 'Male':
        return 0
    else:
        print("ERROR: y cannot be empty in gender")
        return np.NaN


clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])


clinical['GENDER'] = clinical['GENDER'].apply(convert_gender)
clinical['GENDER'] = pd.to_numeric(clinical['GENDER'])

/tmp/ipykernel_266354/2970686832.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
/tmp/ipykernel_266354/2970686832.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])
/tmp/ipykernel_266354/2970686832.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [168]:
len(filtered_clinical) + 12

331

In [169]:
# remove patients wit more than 80 yo and less than 6 months OS
dropped = 0
dropped += len(clinical[((clinical['AGE'] >= 80) & 
                         (clinical['OS_MONTHS'] <= 6))])

dropped += len(clinical[clinical['OS_MONTHS']
               .isna() == True])

filtered_clinical = clinical[~(
    (clinical['AGE'] >= 80) & 
    (clinical['OS_MONTHS'] <= 6))]

filtered_clinical = filtered_clinical[filtered_clinical['OS_MONTHS'].isna() == False]

print("dropped {} rows".format(dropped))

dropped 13 rows


In [170]:
print(microbiome.shape, filtered_clinical.shape)

(332, 1406) (319, 4)


In [171]:
def merge_all(omics):
    df = omics[0]
    starting_indices = [0,omics[0].shape[1]]
    for omic in omics[1:]:
        #print(omic)
        starting_indices.append(starting_indices[-1] + omic.shape[1])
        df = df.merge(omic, left_index=True, right_index=True)
    return (df, starting_indices[:-1])

In [172]:
final_df, indices = merge_all(
    [filtered_clinical, 
     rna_median, 
     metilation, 
     mirna ,
     cna_log2, 
     rppa_levels, 
     alterations,
     microbiome])

In [173]:
indices

[0, 4, 2009, 3550, 4293, 6569, 6758, 9165]

In [174]:
(final_df.shape)

(319, 10571)

In [175]:
pd.DataFrame(indices, columns=['index']).to_csv("DaniDatasets/indices_for_preprocessed.csv", index=False)

In [176]:
final_df = final_df.rename(columns={'OS_STATUS':'event', 'OS_MONTHS': 'time'})

In [177]:
final_df.to_csv("DaniDatasets/preprocessed.csv")

In [178]:
final_df.columns

Index(['AGE', 'GENDER', 'event', 'time', 'rna_median_A1CF', 'rna_median_ABCG2',
       'rna_median_ABI1', 'rna_median_ABI2', 'rna_median_ABL1',
       'rna_median_ABL2',
       ...
       'microbiome_Yokenella', 'microbiome_Yonghaparkia',
       'microbiome_Youngiibacter', 'microbiome_Zavarzinella',
       'microbiome_Zhouia', 'microbiome_Zobellia', 'microbiome_Zooshikella',
       'microbiome_Zunongwangia', 'microbiome_Zymobacter',
       'microbiome_Zymomonas'],
      dtype='object', length=10571)